<a href="https://colab.research.google.com/github/shnhrtkyk/JTCcode/blob/main/04_%E7%89%A9%E4%BD%93%E8%AA%8D%E8%AD%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# クラス分類
点群のクラスを分類する方法を実装します。



## 環境設定

## ライブラリインストール

In [ ]:
!pip install open3d==0.16

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.5/422.5 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 81.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.3/75.3 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 62.4 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.0.0
    Uninstalling Werkzeug-3.0.0:
      Successfully uninstalled Werkzeug-3.0.0
  Attempting uninstall: nbformat
    Found existing installation: nbformat 5.9.2
    Uninstalling nbformat-5.9.2:
      Successfully uninstalled nbformat-5.9.2


## クラス分類の概要
クラス分類では、読み込んだ点群の特徴量を算出し、その特徴量を用いてクラス分類を行う機械学習モデルを学習します。そして、学習済みモデルを用いて未知の点群のクラス分類を実行します。


## データのダウンロード
ここでは、公開データセットを用いてクラス分類を実行します。
[公開データセット](https://rgbd-dataset.cs.washington.edu/dataset/rgbd-dataset_pcd_ascii/)は、51種類のクラスごとに、300個の点群を有するデータです。クラスの例として、りんごなどの果物、カメラなどの一般的な物体などがあります。

以下のコードでは、上記のリンクからデータをダウンロードします。学習の時間を短縮するために、["apple", "banana", "camera"]のみを指定してクラス数を削減しています。


In [ ]:
import os
import open3d as o3d
import numpy as np

dirname = "rgbd-dataset"
classes = ["apple", "banana", "camera"]
url="https://rgbd-dataset.cs.washington.edu/dataset/rgbd-dataset_pcd_ascii/"
for i in range(len(classes)):
    if not os.path.exists(dirname + "/" + classes[i]):
        os.system("wget " + url + classes[i] + "_1.tar")
        os.system("tar xvf " + classes[i] + "_1.tar")



### 表示して確認
ダウンロードした点群を表示します。
まずは、りんごのデータを表示します。拡張子はpcd形式と呼ばれるPoint Cloud Libraryで用いられる形式になっていますが、Open3Dでも読み込めます。

In [ ]:
apple = "/content/rgbd-dataset/apple/apple_1/apple_1_1_3.pcd"
# main

print("Loading a point cloud from", apple)
pcd = o3d.io.read_point_cloud(apple)
print(pcd)


pcd.paint_uniform_color([0.5, 0.5, 0.5])
o3d.visualization.draw_plotly(
  [pcd],
  width=1200,
  height=800
)


Loading a point cloud from /content/rgbd-dataset/apple/apple_1/apple_1_1_3.pcd
PointCloud with 3198 points.


次に、バナナのデータを表示します。

In [ ]:
banana = "/content/rgbd-dataset/banana/banana_1/banana_1_1_15.pcd"
# main

print("Loading a point cloud from", banana)
pcd = o3d.io.read_point_cloud(banana)
print(pcd)


pcd.paint_uniform_color([0.5, 0.5, 0.5])
o3d.visualization.draw_plotly(
  [pcd],
  width=1200,
  height=800
)


Loading a point cloud from /content/rgbd-dataset/banana/banana_1/banana_1_1_15.pcd
PointCloud with 3238 points.


最後に、カメラのデータを表示します。

In [ ]:
camera = "/content/rgbd-dataset/camera/camera_1/camera_1_1_51.pcd"
# main

print("Loading a point cloud from", camera)
pcd = o3d.io.read_point_cloud(camera)
print(pcd)


pcd.paint_uniform_color([0.5, 0.5, 0.5])
o3d.visualization.draw_plotly(
  [pcd],
  width=1200,
  height=800
)


Loading a point cloud from /content/rgbd-dataset/camera/camera_1/camera_1_1_51.pcd
PointCloud with 3692 points.


## 特徴量の算出
点群をクラス分類するため、入力された点群全体に対する特徴量を求める必要があります。点群全体に対する特徴量として、大局特徴量を用います。ここで、大局特徴量はfpfhを用います。


In [ ]:
# fpfhを求める関数
def extract_fpfh( filename ):
    print (" ", filename)
    # 点群の読み込み
    pcd = o3d.io.read_point_cloud(filename)
    # 処理の軽量化のために
    pcd = pcd.voxel_down_sample(0.01)
    # 基本的な情報として法線を計算します
    pcd.estimate_normals(
        search_param = o3d.geometry.KDTreeSearchParamHybrid(radius=0.02, max_nn=10))
    # FPFHの計算
    fpfh = o3d.pipelines.registration.compute_fpfh_feature(pcd,
        search_param = o3d.geometry.KDTreeSearchParamHybrid(radius=0.03, max_nn=100))
    # 0-1の値域に正規化を行います
    sum_fpfh = np.sum(np.array(fpfh.data),1)
    return( sum_fpfh / np.linalg.norm(sum_fpfh) )

# Extract features FPFH
nsamp = 100
# 訓練と評価用のテストに分けます
feat_train = np.zeros( (len(classes), nsamp, 33) )
feat_test = np.zeros( (len(classes), nsamp, 33) )
# 点群ごとに特徴量の計算
for i in range(len(classes)):
    print ("Extracting train features in " + classes[i] + "...")
    for n in range(nsamp):
        filename = dirname + "/" + classes[i] + "/" + classes[i] + \
                   "_1/" + classes[i] + "_1_1_" + str(n+1) + ".pcd"
        feat_train[ i, n ] = extract_fpfh( filename )
    print ("Extracting test features in " + classes[i] + "...")
    for n in range(nsamp):
        filename = dirname + "/" + classes[i] + "/" + classes[i] + \
                   "_1/" + classes[i] + "_1_4_" + str(n+1) + ".pcd"
        feat_test[ i, n ] = extract_fpfh( filename )




Extracting train features in apple...
  rgbd-dataset/apple/apple_1/apple_1_1_1.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_2.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_3.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_4.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_5.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_6.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_7.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_8.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_9.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_10.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_11.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_12.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_13.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_14.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_15.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_16.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_17.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_18.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_19.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_20.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_21.pcd
  rgb

## 分類器で推論


上記の処理で求めた各点群の特徴量を用いて、点群の分類を行います。
SVMなどの機械学習モデルを使用した学習は時間がかかるので、作成した特徴量との類似度を計算して分類します。
類似度の計算には、特徴量の内積を用います。内積は類似している場合には1に近い値を示します。



In [ ]:
# 各クラスとの類似度を計算して、最も類似した特徴量を持つクラスを探す
for i in range(len(classes)):
    max_sim = np.zeros((3, nsamp))
    for j in range(len(classes)):
        # テストデータのクラスと訓練データのクラスで、特徴量どうしの類似度を計算する
        sim = np.dot(feat_test[i], feat_train[j].transpose()) # 内積を計算して
        max_sim[j] = np.max(sim,1) # 一番近い特徴量を探す
    correct_num = (np.argmax(max_sim,0) == i).sum()
    print ("Accuracy of", classes[i], ":", correct_num*100/nsamp, "%")

Accuracy of apple : 98.0 %
Accuracy of banana : 89.0 %
Accuracy of camera : 83.0 %


## より多くのクラスで分類
さきほどの例では、りんご・バナナ・カメラの3クラスの分類を行いました。
では、より多くのクラスで分類を行った場合はどうなるでしょうか。
クラス数を増やして実際に確かめてみましょう。

以下のコードでは、クラス分類を行うために読み込むクラス数を10個に増やします。

In [ ]:
import os
import open3d as o3d
import numpy as np

dirname = "rgbd-dataset"
classes = ["apple", "banana", "camera", "cell_phone", "food_bag", "lemon", "scissors", "sponge", "potato" , "keyboard"]
url="https://rgbd-dataset.cs.washington.edu/dataset/rgbd-dataset_pcd_ascii/"
for i in range(len(classes)):
    if not os.path.exists(dirname + "/" + classes[i]):
        os.system("wget " + url + classes[i] + "_1.tar")
        os.system("tar xvf " + classes[i] + "_1.tar")



特徴量の計算

In [ ]:
def extract_fpfh( filename ):
    print (" ", filename)
    pcd = o3d.io.read_point_cloud(filename)
    pcd = pcd.voxel_down_sample(0.01)
    pcd.estimate_normals(
        search_param = o3d.geometry.KDTreeSearchParamHybrid(radius=0.02, max_nn=10))
    fpfh = o3d.pipelines.registration.compute_fpfh_feature(pcd,
        search_param = o3d.geometry.KDTreeSearchParamHybrid(radius=0.03, max_nn=100))
    sum_fpfh = np.sum(np.array(fpfh.data),1)
    return( sum_fpfh / np.linalg.norm(sum_fpfh) )

# Extract features FPFH
nsamp = 100
feat_train = np.zeros( (len(classes), nsamp, 33) )
feat_test = np.zeros( (len(classes), nsamp, 33) )
for i in range(len(classes)):
    print ("Extracting train features in " + classes[i] + "...")
    for n in range(nsamp):
        filename = dirname + "/" + classes[i] + "/" + classes[i] + \
                   "_1/" + classes[i] + "_1_1_" + str(n+1) + ".pcd"
        feat_train[ i, n ] = extract_fpfh( filename )
    print ("Extracting test features in " + classes[i] + "...")
    for n in range(nsamp):
        filename = dirname + "/" + classes[i] + "/" + classes[i] + \
                   "_1/" + classes[i] + "_1_4_" + str(n+1) + ".pcd"
        feat_test[ i, n ] = extract_fpfh( filename )




Extracting train features in apple...
  rgbd-dataset/apple/apple_1/apple_1_1_1.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_2.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_3.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_4.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_5.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_6.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_7.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_8.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_9.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_10.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_11.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_12.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_13.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_14.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_15.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_16.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_17.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_18.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_19.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_20.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_21.pcd
  rgb

それでは実際に、クラス分類を行います。
Accuracyの値について、クラス数を10に増やした場合と、クラス数が3の場合を比較してみましょう。

In [ ]:
# 1-NN classification
for i in range(len(classes)):
    max_sim = np.zeros((10, nsamp))
    for j in range(len(classes)):
        sim = np.dot(feat_test[i], feat_train[j].transpose()) # 内積を計算して
        max_sim[j] = np.max(sim,1) # 一番近い特徴量を探す
    correct_num = (np.argmax(max_sim,0) == i).sum()
    print ("Accuracy of", classes[i], ":", correct_num*100/nsamp, "%")

Accuracy of apple : 42.0 %
Accuracy of banana : 73.0 %
Accuracy of camera : 54.0 %
Accuracy of cell_phone : 20.0 %
Accuracy of food_bag : 70.0 %
Accuracy of lemon : 67.0 %
Accuracy of scissors : 17.0 %
Accuracy of sponge : 5.0 %
Accuracy of potato : 93.0 %
Accuracy of keyboard : 0.0 %


3クラスのときよりも分類性能が悪くなったことが確認できます。
そのため、特徴量の作成と簡単な機械学習モデルでは限界があると言えます。
したがって、より高性能な手法として、深層学習のようなデータから特徴を獲得するような手法が有効ではないかと考えられます。
